In [ ]:
def install_requirements():
  !pip install segmentation-models-pytorch
  !pip install wandb

def connect_to_drive():
  from google.colab import drive
  drive.mount('/content/gdrive', force_remount=True)

def split_dataset_into_train_and_valid(dataset, percentage_train):
  lengths = [int(len(dataset)*percentage_train), len(dataset)-int(len(dataset)*percentage_train)]
  train_dataset, valid_dataset = random_split(dataset, lengths)
  return train_dataset, valid_dataset

def save_paths_images(list_paths, title):
  f  = open(title, "w")
  for path in list_paths:
    f.write("%s" % str(path)+'\n')
  f.close()

def create_list_images_paths_from_folder(input_path_training, input_path_test):
  imgs_training = [path for path in sorted(Path(input_path_training).glob("**/*.png")) if 'seg' not in str(path).split('/')[-1]]
  random.shuffle(imgs_training)
  training_imgs_list = imgs_training[:160]
  validation_imgs_list = imgs_training[160:]
  test_imgs_list = [path for path in sorted(Path(input_path_test).glob("**/*.png")) if 'seg' not in str(path).split('/')[-1]]
  return training_imgs_list, validation_imgs_list, test_imgs_list

def read_list_images_path_from_txt(txt_path):
  with open(txt_path) as f:
    content = f.readlines()
    content = [x.strip() for x in content]
    return content
  f.close()

import matplotlib.pyplot as plt

def visualize_image_label_overlay(img, label, alpha):
  '''
  img is a tensor, label is a numpy array
  '''
  plt.imshow(img.permute(1,2,0))
  plt.imshow(label.transpose(1,2,0).squeeze(), cmap='Greys', alpha=alpha)
  plt.show()


colour_dictionary={
    'white':torch.Tensor([255,255,255]),
    'green':torch.Tensor([118,246,165]),
    'red': torch.Tensor([245,119,65]),
    'blue': torch.Tensor([107,136,250])
}

def visualize_prediction(prediction, label, colour_dictionary):
  '''
  prediction and label are two numpy arrays in two dimensions WxH
  White=TN, Green=TP, RED=FP, BLUE=FN
  '''
  visualization=np.empty(shape=(prediction.shape[0],prediction.shape[1],3))
  for i in range(prediction.shape[0]):
    for j in range(prediction.shape[1]):
      if prediction[i][j]==1 and label[i][j]==1: #TP
        visualization[i,j,:]=colour_dictionary['green']
      if prediction[i][j]==0 and label[i][j]==0: #TN
        visualization[i,j,:]=colour_dictionary['white']
      if prediction[i][j]==0 and label[i][j]==1: #FN
        visualization[i,j,:]=colour_dictionary['blue']
      if prediction[i][j]==1 and label[i][j]==0: #FP
        visualization[i,j,:]=colour_dictionary['red']
  plt.imshow(visualization.astype(np.uint8))
